In [69]:
import time

# getpass - to input our password without showing it in the notebook
from getpass import getpass

# Juicy stuff - these are the Classes we will use for interaction with a webpage:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common import NoSuchElementException, ElementClickInterceptedException

# libraries for interacting with the operating system (OS)
import pathlib
import os
from os.path import join

import pandas as pd
import random
import re
from bs4 import BeautifulSoup
import re

import warnings
warnings.filterwarnings('ignore')

import numpy as np


In [70]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


In [72]:
def perform_search_and_scroll(driver):
    try:
        max_attempts = 10
        attempts = 0
        while attempts < max_attempts:
            try:
                button_search = driver.find_element(By.CLASS_NAME, 'zoeken-resultaten-lijst-meer')
                button_search.click()
            except:
                attempts += 1

        # Scroll to the end of the page
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)

        # Optionally, you can add a delay to give the page time to load new content
        # import time
        # time.sleep(2)  # 2 seconds of delay (adjust as needed)
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [73]:
def extract_school_details (url):
    driver.get(url)
    
    #cookies
    import pickle
    cookies_dir = 'saved_cookies'
    lis_dir = os.listdir()

    if cookies_dir not in lis_dir:
        os.mkdir(cookies_dir)
    else:
        pass # os.removedirs(cookies_dir) --> to remove a directory

    save_location = cookies_dir + '/cookies.pkl'
    pickle.dump(driver.get_cookies() , open(save_location,"wb"))
    
    max_attempts = 10 
    attempts = 0

    while attempts < max_attempts:
        try:
            button_search = driver.find_element(By.CLASS_NAME, 'zoeken-resultaten-lijst-meer')
            button_search.click()
               
        except:
            attempts += 1
               
    
    #getting data
    html = driver.page_source
    soup = BeautifulSoup(html)
    soup.find_all(attrs={'class': re.compile(r'zoekresultaat-title')})
    
    #links
    school_links_dirty = soup.select('h3 a')
    school_links_clean = [item.get('href') for item in school_links_dirty]
    
    #names
    school_list_dirty = soup.find_all(attrs={'class': re.compile(r'zoekresultaat-title')})
    school_list_clean = [school.text.strip() for school in school_list_dirty]
    
    #adresses
    adress_list_dirty = soup.find_all(attrs={'class': re.compile(r'zoekresultaat-adres')})
    adress_list_clean = [job.text.strip() for job in adress_list_dirty]
    
    #details
    details_list_dirty = soup.find_all(attrs={'class': re.compile(r'zoekresultaat-details')})
    details_list_clean = [job.text.strip() for job in details_list_dirty]
    
    #number of students
    details_list_clean
    pattern = r'\d+'
    number_of_students = [re.search(pattern, item).group() if re.search(pattern, item) else '' for item in details_list_clean]
        
    return pd.DataFrame({'name':school_list_clean, 'address':adress_list_clean, 'link':school_links_clean, 'details':details_list_clean, 'n_students':number_of_students})

In [74]:
def create_database (csv):    
    mschool_links = pd.read_csv(csv)
    dfs_list = []
    for ind in mschool_links.index:
        df = (extract_school_details(mschool_links['Link'][ind]))
        df['neighborhood'] = mschool_links['Name'][ind]
        dfs_list.append(df)
    final_df = pd.concat(dfs_list, ignore_index=True)
    return final_df

In [75]:
final_mschool_df = create_database('places_middle_schools.csv')

In [76]:
final_mschool_df['type'] = 'Middle School'
final_mschool_df

,name,address,link,details,n_students,neighborhood,type
0,Yuverta vmbo Aalsmeer,Jac. P. Thijsselaan 18,/middelbare-scholen/aalsmeer/1713/yuverta-vmbo...,OverigeVoortgezet onderwijs535 leerlingen,535,Aalsmeer,Middle School
1,Yuverta mavo Aalsmeer,1e J.C. Mensinglaan 40,/middelbare-scholen/aalsmeer/1717/yuverta-mavo...,OverigeVoortgezet onderwijs491 leerlingen,491,Aalsmeer,Middle School
2,Chr Coll Schaersvoorde,Slingelaan 28,/middelbare-scholen/aalten/3289/chr-coll-schae...,Prot. christelijkVoortgezet onderwijs1019 leer...,1019,Aalten,Middle School
3,Chr Coll Schaersvoorde,Landbouwstraat 1,/middelbare-scholen/aalten/23137/chr-coll-scha...,Prot. christelijkVoortgezet onderwijs382 leerl...,382,Aalten,Middle School
4,Montessori College Aerdenhout,Boekenroodeweg 5,/middelbare-scholen/aerdenhout/2536/montessori...,SamenwerkingVoortgezet onderwijs427 leerlingen...,427,Aerdenhout,Middle School
...,...,...,...,...,...,...,...
1796,Thorbecke Scholengemeenschap,Dokter van Heesweg 1,/middelbare-scholen/zwolle/22498/thorbecke-sch...,OpenbaarVoortgezet onderwijs1465 leerlingen,1465,Zwolle,Middle School
1797,Thorbecke Scholengemeenschap voor Ath Havo Mav...,Russenweg 3,/middelbare-scholen/zwolle/22508/thorbecke-sch...,OpenbaarVoortgezet onderwijs604 leerlingen,604,Zwolle,Middle School
1798,Thorbecke SGM voor Ath Havo Mavo Vbo Lwoo,Sleedoornstraat 1,/middelbare-scholen/zwolle/26556/thorbecke-sgm...,OverigeVoortgezet onderwijs15 leerlingen,15,Zwolle,Middle School
1799,Greijdanus - Zwolle,Campus 5,/middelbare-scholen/zwolle/2869/greijdanus-zwo...,Prot. christelijkVoortgezet onderwijs2345 leer...,2345,Zwolle,Middle School


In [77]:
final_bschool_df = create_database('places_basic_schools.csv')

In [78]:
final_bschool_df['type'] = 'Basic School'
final_bschool_df

,name,address,link,details,n_students,neighborhood,type
0,Kindcentrum De Wegwijzer,Harm Tiesingstraat 74,/basisscholen/2e-exloërmond/25159/kindcentrum-...,Prot. christelijkBasisonderwijs84 leerlingen,84,2e Exloërmond,Basic School
1,Openbare Basisschool De Westhoek,Schoolstraat 27,/basisscholen/2e-exloërmond/7595/openbare-basi...,OpenbaarBasisonderwijs44 leerlingen,44,2e Exloërmond,Basic School
2,Basisschool De Tweeklank,Albardastraat 35,/basisscholen/aadorp/11785/basisschool-de-twee...,SamenwerkingBasisonderwijs116 leerlingenDalton,116,Aadorp,Basic School
3,JHR Willem Versluijsschool,Prelaatweg 22,/basisscholen/aagtekerke/25963/jhr-willem-vers...,Prot. christelijkBasisonderwijs191 leerlingen,191,Aagtekerke,Basic School
4,Christelijke Basisschool de Schutse,Paardelandsdrift 9,/basisscholen/aalden/9661/christelijke-basissc...,Prot. christelijkBasisonderwijs62 leerlingen,62,Aalden,Basic School
...,...,...,...,...,...,...,...
7045,Buitenwijs,Nilantsweg 77,/basisscholen/zwolle/26866/buitenwijs/,OverigeBasisonderwijs,,Zwolle,Basic School
7046,Kentalis Enkschool SO Zwolle,Jan Buschstraat 6,/basisscholen/zwolle/23943/kentalis-enkschool-...,Algemeen bijzonderSpeciaal onderwijs158 leerli...,158,Zwolle,Basic School
7047,Michaelschool,Bachlaan 8,/basisscholen/zwolle/8415/michaelschool/,Algemeen bijzonderBasisonderwijs212 leerlingen...,212,Zwolle,Basic School
7048,Ridder Jorisschool,Korianderplein\t\t 8-10,/basisscholen/zwolle/24895/ridder-jorisschool/,Algemeen bijzonderBasisonderwijs187 leerlingen...,187,Zwolle,Basic School


In [79]:
df_final = pd.concat([final_mschool_df,final_bschool_df], ignore_index=True)
df_final

,name,address,link,details,n_students,neighborhood,type
0,Yuverta vmbo Aalsmeer,Jac. P. Thijsselaan 18,/middelbare-scholen/aalsmeer/1713/yuverta-vmbo...,OverigeVoortgezet onderwijs535 leerlingen,535,Aalsmeer,Middle School
1,Yuverta mavo Aalsmeer,1e J.C. Mensinglaan 40,/middelbare-scholen/aalsmeer/1717/yuverta-mavo...,OverigeVoortgezet onderwijs491 leerlingen,491,Aalsmeer,Middle School
2,Chr Coll Schaersvoorde,Slingelaan 28,/middelbare-scholen/aalten/3289/chr-coll-schae...,Prot. christelijkVoortgezet onderwijs1019 leer...,1019,Aalten,Middle School
3,Chr Coll Schaersvoorde,Landbouwstraat 1,/middelbare-scholen/aalten/23137/chr-coll-scha...,Prot. christelijkVoortgezet onderwijs382 leerl...,382,Aalten,Middle School
4,Montessori College Aerdenhout,Boekenroodeweg 5,/middelbare-scholen/aerdenhout/2536/montessori...,SamenwerkingVoortgezet onderwijs427 leerlingen...,427,Aerdenhout,Middle School
...,...,...,...,...,...,...,...
8846,Buitenwijs,Nilantsweg 77,/basisscholen/zwolle/26866/buitenwijs/,OverigeBasisonderwijs,,Zwolle,Basic School
8847,Kentalis Enkschool SO Zwolle,Jan Buschstraat 6,/basisscholen/zwolle/23943/kentalis-enkschool-...,Algemeen bijzonderSpeciaal onderwijs158 leerli...,158,Zwolle,Basic School
8848,Michaelschool,Bachlaan 8,/basisscholen/zwolle/8415/michaelschool/,Algemeen bijzonderBasisonderwijs212 leerlingen...,212,Zwolle,Basic School
8849,Ridder Jorisschool,Korianderplein\t\t 8-10,/basisscholen/zwolle/24895/ridder-jorisschool/,Algemeen bijzonderBasisonderwijs187 leerlingen...,187,Zwolle,Basic School


In [80]:
df_final.to_csv('final_allschools_df2.csv')

In [81]:
import pandas as pd
data = pd.read_csv("final_allschools_df2.csv")
data

,Unnamed: 0,name,address,link,details,n_students,neighborhood,type
0,0,Yuverta vmbo Aalsmeer,Jac. P. Thijsselaan 18,/middelbare-scholen/aalsmeer/1713/yuverta-vmbo...,OverigeVoortgezet onderwijs535 leerlingen,535.0,Aalsmeer,Middle School
1,1,Yuverta mavo Aalsmeer,1e J.C. Mensinglaan 40,/middelbare-scholen/aalsmeer/1717/yuverta-mavo...,OverigeVoortgezet onderwijs491 leerlingen,491.0,Aalsmeer,Middle School
2,2,Chr Coll Schaersvoorde,Slingelaan 28,/middelbare-scholen/aalten/3289/chr-coll-schae...,Prot. christelijkVoortgezet onderwijs1019 leer...,1019.0,Aalten,Middle School
3,3,Chr Coll Schaersvoorde,Landbouwstraat 1,/middelbare-scholen/aalten/23137/chr-coll-scha...,Prot. christelijkVoortgezet onderwijs382 leerl...,382.0,Aalten,Middle School
4,4,Montessori College Aerdenhout,Boekenroodeweg 5,/middelbare-scholen/aerdenhout/2536/montessori...,SamenwerkingVoortgezet onderwijs427 leerlingen...,427.0,Aerdenhout,Middle School
...,...,...,...,...,...,...,...,...
8846,8846,Buitenwijs,Nilantsweg 77,/basisscholen/zwolle/26866/buitenwijs/,OverigeBasisonderwijs,NaN,Zwolle,Basic School
8847,8847,Kentalis Enkschool SO Zwolle,Jan Buschstraat 6,/basisscholen/zwolle/23943/kentalis-enkschool-...,Algemeen bijzonderSpeciaal onderwijs158 leerli...,158.0,Zwolle,Basic School
8848,8848,Michaelschool,Bachlaan 8,/basisscholen/zwolle/8415/michaelschool/,Algemeen bijzonderBasisonderwijs212 leerlingen...,212.0,Zwolle,Basic School
8849,8849,Ridder Jorisschool,Korianderplein\t\t 8-10,/basisscholen/zwolle/24895/ridder-jorisschool/,Algemeen bijzonderBasisonderwijs187 leerlingen...,187.0,Zwolle,Basic School


In [82]:
data.groupby('type').nunique('neighborhood')

,Unnamed: 0,name,address,link,details,n_students,neighborhood
type,,,,,,,
Basic School,7050,6329,6788,7049,3269,619,1753
Middle School,1801,1720,1698,1801,1595,957,407


In [83]:
data.drop_duplicates(subset='link', keep='first', inplace=True)

In [84]:
data

,Unnamed: 0,name,address,link,details,n_students,neighborhood,type
0,0,Yuverta vmbo Aalsmeer,Jac. P. Thijsselaan 18,/middelbare-scholen/aalsmeer/1713/yuverta-vmbo...,OverigeVoortgezet onderwijs535 leerlingen,535.0,Aalsmeer,Middle School
1,1,Yuverta mavo Aalsmeer,1e J.C. Mensinglaan 40,/middelbare-scholen/aalsmeer/1717/yuverta-mavo...,OverigeVoortgezet onderwijs491 leerlingen,491.0,Aalsmeer,Middle School
2,2,Chr Coll Schaersvoorde,Slingelaan 28,/middelbare-scholen/aalten/3289/chr-coll-schae...,Prot. christelijkVoortgezet onderwijs1019 leer...,1019.0,Aalten,Middle School
3,3,Chr Coll Schaersvoorde,Landbouwstraat 1,/middelbare-scholen/aalten/23137/chr-coll-scha...,Prot. christelijkVoortgezet onderwijs382 leerl...,382.0,Aalten,Middle School
4,4,Montessori College Aerdenhout,Boekenroodeweg 5,/middelbare-scholen/aerdenhout/2536/montessori...,SamenwerkingVoortgezet onderwijs427 leerlingen...,427.0,Aerdenhout,Middle School
...,...,...,...,...,...,...,...,...
8845,8845,IKC Het Festival,Sportlaan 6,/basisscholen/zwolle/22951/ikc-het-festival/,OpenbaarBasisonderwijs681 leerlingen,681.0,Zwolle,Basic School
8846,8846,Buitenwijs,Nilantsweg 77,/basisscholen/zwolle/26866/buitenwijs/,OverigeBasisonderwijs,NaN,Zwolle,Basic School
8847,8847,Kentalis Enkschool SO Zwolle,Jan Buschstraat 6,/basisscholen/zwolle/23943/kentalis-enkschool-...,Algemeen bijzonderSpeciaal onderwijs158 leerli...,158.0,Zwolle,Basic School
8848,8848,Michaelschool,Bachlaan 8,/basisscholen/zwolle/8415/michaelschool/,Algemeen bijzonderBasisonderwijs212 leerlingen...,212.0,Zwolle,Basic School


In [85]:
print(data['type'].value_counts()['Middle School'])

1801


In [86]:
print(data['type'].value_counts()['Basic School'])

7049


In [87]:
data_reset= data.reset_index(drop=True)
data_reset

,Unnamed: 0,name,address,link,details,n_students,neighborhood,type
0,0,Yuverta vmbo Aalsmeer,Jac. P. Thijsselaan 18,/middelbare-scholen/aalsmeer/1713/yuverta-vmbo...,OverigeVoortgezet onderwijs535 leerlingen,535.0,Aalsmeer,Middle School
1,1,Yuverta mavo Aalsmeer,1e J.C. Mensinglaan 40,/middelbare-scholen/aalsmeer/1717/yuverta-mavo...,OverigeVoortgezet onderwijs491 leerlingen,491.0,Aalsmeer,Middle School
2,2,Chr Coll Schaersvoorde,Slingelaan 28,/middelbare-scholen/aalten/3289/chr-coll-schae...,Prot. christelijkVoortgezet onderwijs1019 leer...,1019.0,Aalten,Middle School
3,3,Chr Coll Schaersvoorde,Landbouwstraat 1,/middelbare-scholen/aalten/23137/chr-coll-scha...,Prot. christelijkVoortgezet onderwijs382 leerl...,382.0,Aalten,Middle School
4,4,Montessori College Aerdenhout,Boekenroodeweg 5,/middelbare-scholen/aerdenhout/2536/montessori...,SamenwerkingVoortgezet onderwijs427 leerlingen...,427.0,Aerdenhout,Middle School
...,...,...,...,...,...,...,...,...
8845,8845,IKC Het Festival,Sportlaan 6,/basisscholen/zwolle/22951/ikc-het-festival/,OpenbaarBasisonderwijs681 leerlingen,681.0,Zwolle,Basic School
8846,8846,Buitenwijs,Nilantsweg 77,/basisscholen/zwolle/26866/buitenwijs/,OverigeBasisonderwijs,NaN,Zwolle,Basic School
8847,8847,Kentalis Enkschool SO Zwolle,Jan Buschstraat 6,/basisscholen/zwolle/23943/kentalis-enkschool-...,Algemeen bijzonderSpeciaal onderwijs158 leerli...,158.0,Zwolle,Basic School
8848,8848,Michaelschool,Bachlaan 8,/basisscholen/zwolle/8415/michaelschool/,Algemeen bijzonderBasisonderwijs212 leerlingen...,212.0,Zwolle,Basic School
